In [3]:
import torch
import torch.nn as nn


In [4]:
# Model 선언
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear1 = nn.Linear(2,3,bias=False)
        self.linear2 = nn.Linear(3,2,bias=False)
        self.linear3 = nn.Linear(2,1,bias=False)
        

    def forward(self, x):
        x = self.linear1(x)
        x = self.linear2(x)
        x = self.linear3(x)
        return x

# model의 weight 전부 0.5로 초기화하는 함수 (이해를 돕기 위하여)
def init_weights(m):
    if type(m) == nn.Linear:
        m.weight.data.fill_(0.5)

def print_grad(name):
    def hook(model, input, output):
        print('BACKWARD', name, input, output)
    return hook

def preprocessing(name):
    def hook(model, input):
        print('못 믿으시겠다면', input)
        input[0][:2] = 1000
    return hook

def print_forward(name):
    def hook(model, input, output):
        print("FORWARD", name, input, output)
    return hook

# def 함수 ( 변수):
#     def hook(주의할 부분):
#         ***************** // 여기가 너가 만들어야 하는 부분이고.
#     return hook

# model instance 생성 및 weight 초기화 


In [12]:

model = Model()
model.apply(init_weights)

param_list = []

for name, parameter in model.named_parameters():
    print(f'Parameter : ', name)
    if name == 'linear2.weight':
        param_list.append(parameter)

for name, module in model.named_modules():
    print(f'Module : ', name)
    module.register_forward_hook(print_forward(name))

    if name == 'linear2':  
        module.register_forward_pre_hook(preprocessing(name))
    
        #module.register_full_backward_hook(print_grad(name))


    #module.register_forward_hook(함수(변수))

optimizer = torch.optim.SGD(param_list, lr=1)
# optimizer = torch.optim.SGD(model.parameters(), lr=1)

for iteration in range(1):
    print('')
    
    for name, parameter in model.named_parameters():
        print(name, parameter)
    print('')
    x = torch.FloatTensor([1,4])
    print("x is",x)
    y = model(x)

    loss  = (y - 2) ** 2

    print('')
    print(y) ########### y 
    optimizer.zero_grad()
    print('')
    print('<<<<<<<<<<<back')
    loss.backward()
    print('<<<<<<<<<<<done')
    optimizer.step()
    print('')
    for name, parameter in model.named_parameters():
        print(name, parameter)


Parameter :  linear1.weight
Parameter :  linear2.weight
Parameter :  linear3.weight
Module :  
Module :  linear1
Module :  linear2
Module :  linear3

linear1.weight Parameter containing:
tensor([[0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000]], requires_grad=True)
linear2.weight Parameter containing:
tensor([[0.5000, 0.5000, 0.5000],
        [0.5000, 0.5000, 0.5000]], requires_grad=True)
linear3.weight Parameter containing:
tensor([[0.5000, 0.5000]], requires_grad=True)

x is tensor([1., 4.])
FORWARD linear1 (tensor([1., 4.]),) tensor([2.5000, 2.5000, 2.5000], grad_fn=<SqueezeBackward3>)
못 믿으시겠다면 (tensor([2.5000, 2.5000, 2.5000], grad_fn=<SqueezeBackward3>),)
FORWARD linear2 (tensor([1000.0000, 1000.0000,    2.5000], grad_fn=<CopySlices>),) tensor([1001.2500, 1001.2500], grad_fn=<SqueezeBackward3>)
FORWARD linear3 (tensor([1001.2500, 1001.2500], grad_fn=<SqueezeBackward3>),) tensor([1001.2500], grad_fn=<SqueezeBackward3>)
FORWARD  (tensor([1., 4.]),) tensor([1001.2